In [ ]:
import ipywidgets as widgets
from IPython.display import Latex
import re
from typing import Optional

def addConstraint(b):
    st.append(widgets.Text(placeholder="Vincolo {}".format(len(st)+1)))
    with output:
        display(st[-1])

of:widgets.Text = widgets.Text(placeholder="Funzione obiettivo",description="min")
output:widgets.Output = widgets.Output()
st:list[widgets.Text] = []
addSt:widgets.Button = widgets.Button(description="Aggiungi vincolo")
addSt.on_click(addConstraint)
display(widgets.Label(value="Inserisci funzione obiettivo in forma di minimo e vincoli, si assumono tutte le variabili positive"))
display(widgets.Label(value="Tutte le equazioni vanno scritte secondo la notazione Latex, ad esempio:"))
display(Latex("3x_1+\\frac{2}{5}x_2=5 diventa $3x_1+\\frac{2}{5}x_2=5$"))
display(widgets.Label(value="Tutte le variabili si devono chiamare x_n con n numero intero"))
display(of)
display(widgets.Label(value="Premi per inserire un nuovo vincolo, si accettano solo problemi in forma standard"))
display(addSt, output)

Label(value='Inserisci funzione obiettivo in forma di minimo e vincoli, si assumono tutte le variabili positiv…

Label(value='Tutte le equazioni vanno scritte secondo la notazione Latex, ad esempio:')

<IPython.core.display.Latex object>

Label(value='Tutte le variabili si devono chiamare x_n con n numero intero')

Text(value='', description='min', placeholder='Funzione obiettivo')

Label(value='Premi per inserire un nuovo vincolo, si accettano solo problemi in forma standard')

Button(description='Aggiungi vincolo', style=ButtonStyle())

Output()

La cella di sotto è per controllare se l'input è corretto

In [ ]:
output:str = "$\\min" + of.value + "$<br>s.t.<br>"
for i in st:
    output +="$"+ i.value + "$<br>"
display(Latex(output))

Eseguire questa cella almeno una volta prima di quella sotto

In [48]:
var_pattern:re.Pattern = re.compile("(?:[+]?(-?[^x]*)x_([^-+=]*))")
def parse_variables_of(f:str):
    variables:list[tuple[int,str | int]] = []

    find:Optional[re.Match[str]] = var_pattern.search(f)
    while (find is not None):
        variables.append((int(find.groups(0)[1]), find.groups(0)[0]))
        find = var_pattern.search(f, pos=find.end())

    max:int = 0
    for i in variables: 
        if i[0] > max: max = i[0]
    out:list[str] = [''] * max
    for i in variables:
        out[i[0]-1] = str(i[1])
    for i in range(len(out)):
        if out[i] is None: out[i] = "0"
    return out

def parse_variables_st(s:list[widgets.Text]):
    variables:list[list[tuple[int,str]]] = []
    kv_pattern:re.Pattern = re.compile(".*=(?P<kv>.*?)$")

    for i in s:
        temp:list[tuple[int, str]] = []
        find:Optional[re.Match[str]] = var_pattern.search(i.value)
        while (find is not None):
            t:str = str(find.groups(0)[0])
            if t=='': t = "1"
            elif t=='-': t ="-1"
            temp.append((int(find.groups(0)[1]), t))
            find = var_pattern.search(i.value, pos=find.end())
        variables.append(temp)
    
    max:int = 0
    for i in variables:
        for j in i:
            if j[0] > max: max = j[0]
    out:list[list[str]] = [['' for _ in range(max+1)] for _ in range(len(s))]
    for i in range(len(out)):
        t2:re.Match[str] | None = kv_pattern.match(s[i].value)
        if (t2 is not None): out[i][max] = t2.group("kv")
    for i in range(len(variables)):
        for j in range(len(variables[i])):
            out[i][variables[i][j][0]-1] = variables[i][j][1]
    for i in range(len(out)):
        for j in range(len(out[i])):
            if out[i][j] == '': out[i][j] = "0"
    return out

def output_tableau(t:list[list[str]]):
    out:str = "<table>"
    for i in t:
        out += "<tr>"
        for j in i:
            out += "<td>\n\n$" + j.replace("'",'').replace("\\\\",'\\') + "$\n</td>\n"
        out += "</tr>"
    out += "</table>"
    display(Latex(out))

def find_id(t:list[list[str]]):
    cols:list[tuple[int, int]] = []
    for i in range(len(t[0])-1):
        if t[0][i] == "0":
            j:int = 1
            id_col:bool = True
            loc_one:tuple[int, int] | None = None
            while (j<len(t) and id_col):
                if (not (t[j][i] == "1" or t[j][i] == "0")): 
                    id_col = False
                    break
                elif (t[j][i] == "1" and loc_one is None): loc_one = (j,i)
                elif (t[j][i] == "1" and loc_one is not None): id_col = False
                j += 1
            if(id_col and loc_one is not None):
                dup = False
                for c in cols:
                    if c[0] == loc_one[0]: 
                        dup = True
                        break
                if not dup : cols.append(loc_one)
    return cols if len(cols)==len(t)-1 else None

Questa cella risolve il problema

In [49]:
for i in st:
    if '\\geq' in i.value or '\\leq' in i.value:
        raise Exception("Problema non in forma standard")

first_row:list[str] = []
other_rows:list[list[str]] = []
first_row = parse_variables_of(of.value)
other_rows = parse_variables_st(st)

lf:int = len(first_row)
lo:int = len(other_rows[0]) - 1
if lf > lo:
    first_row.append("0")
    for i in range(len(other_rows)):
        for _ in range(lo, lf):
            other_rows[i].append("0")
        other_rows[i][-1] = other_rows[i][lo]
        other_rows[i][lo] = "0"
else:
    for _ in range(lf, lo+1): first_row.append("0")

tableau:list[list[str]] = []
tableau.append(first_row)
row: list[str]
for row in other_rows: tableau.append(row)

output_tableau(tableau)
print(find_id(tableau))

<IPython.core.display.Latex object>

[(1, 3), (2, 4), (3, 5)]
